In [1]:
import pandas as pd
import urllib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from inscriptis import get_text
import urllib.request
import urllib3
import re

Using TensorFlow backend.


In [2]:
comments = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('attack_annotations.tsv',  sep = '\t')

In [3]:
comments = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('attack_annotations.tsv',  sep = '\t')

In [3]:
len(annotations['rev_id'].unique())

115864

In [4]:
# labels a comment as an atack if the majority of annoatators did so
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

In [5]:
# join labels and comments
comments['attack'] = labels

In [6]:
# remove newline and tab tokens
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

In [7]:
x=comments.query('attack')['comment']
len(comments.query('attack')['comment'])

13590

In [8]:
# fit a simple text classifier

train_comments = comments.query("split=='train'")
test_comments = comments.query("split=='test'")

clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000, ngram_range = (1,2))),
    ('tfidf', TfidfTransformer(norm = 'l2')),
    ('clf', LogisticRegression()),
])
clf = clf.fit(train_comments['comment'], train_comments['attack'])
auc = roc_auc_score(test_comments['attack'], clf.predict_proba(test_comments['comment'])[:, 1])
print('Test ROC AUC: %.3f' %auc)

C:\Users\islam\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test ROC AUC: 0.957


In [10]:
# correctly classify nice comment
clf.predict(['You have been working hard. You have done a great job'])

array([False])

In [11]:
# correctly classify bad comment
clf.predict(['You are so stupid, only stupid can work like this !'])

array([ True])

In [12]:
from empath import Empath
lexicon = Empath()

In [13]:
html = urllib.request.urlopen("https://sentence.yourdictionary.com/hate").read().decode('utf-8')
val = input("Enter your value: ") 
print(val)
print("=============================================")
text = get_text(html)
text=text.strip()
text=text.split('.')
for i in text:
    i=i.replace("NEWLINE_TOKEN", " ")
    i=i.rstrip()
    for x in range(50):
        i=i.replace(str(x), " ")
    i=i.replace("TAB_TOKEN", " ")
    i=i.replace('[/(){}\[\]\|@,;]','')
    i=i.replace('[^0-9a-z #+_]','')
    i=i.replace('*',' ')
    i="".join([s for s in i.strip().splitlines(True) if s.strip()])

    
    if clf.predict([i])[0]==True:
        print('Detected hate sentance: '+i)
        print('Positive Emotion: '+str(lexicon.analyze(i, categories=["positive_emotion"], normalize=True)['positive_emotion']))
        print('Negative Emotion: '+str(lexicon.analyze(i, categories=["negative_emotion"], normalize=True)['negative_emotion']))
        print('\n ')


Enter your value: https://sentence.yourdictionary.com/hate
https://sentence.yourdictionary.com/hate
Detected hate sentance: I hate those little trolls
Positive Emotion: 0.0
Negative Emotion: 0.2

 
Detected hate sentance: A man offended you and you shot him, and you say you do not know God and hate your life
Positive Emotion: 0.0
Negative Emotion: 0.05

 
Detected hate sentance: Hate it, hate it, hate it, and I can't stop eating it!
                        God, I hate men!
                        They hate my guts
Positive Emotion: 0.0
Negative Emotion: 0.25

 
Detected hate sentance: You must hate me
Positive Emotion: 0.0
Negative Emotion: 0.25

 
Detected hate sentance: I don't hate you, Deidre
Positive Emotion: 0.0
Negative Emotion: 0.2

 
Detected hate sentance: "You don't hate me, and I'm dying soon," she said ruefully
Positive Emotion: 0.0
Negative Emotion: 0.18181818181818182

 
Detected hate sentance: I hate the bastard
Positive Emotion: 0.0
Negative Emotion: 0.25

 
Detected h